In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import requests
import numpy as np
from sklearn import metrics 

Using TensorFlow backend.


In [7]:
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

In [8]:
df.head(5) 

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [9]:
cars = df['name'] 

In [10]:
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values 

In [12]:
def create_model():
    model = Sequential()
    model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dense(10, activation='relu')) # Hidden 2
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam' , metrics = ["accuracy"])
    return model 
    

In [16]:
seed = 7 
np.random.seed(seed) 
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100] 

In [18]:
model = KerasClassifier(build_fn=create_model, verbose=0) 
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x, y)   

C:\Users\User\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [19]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.067839 using {'batch_size': 20, 'epochs': 100}
0.037688 (0.053199) with: {'batch_size': 10, 'epochs': 10}
0.030151 (0.042559) with: {'batch_size': 10, 'epochs': 50}
0.065327 (0.092212) with: {'batch_size': 10, 'epochs': 100}
0.037688 (0.053199) with: {'batch_size': 20, 'epochs': 10}
0.047739 (0.067386) with: {'batch_size': 20, 'epochs': 50}
0.067839 (0.095759) with: {'batch_size': 20, 'epochs': 100}
0.067839 (0.095759) with: {'batch_size': 40, 'epochs': 10}
0.037688 (0.053199) with: {'batch_size': 40, 'epochs': 50}
0.047739 (0.067386) with: {'batch_size': 40, 'epochs': 100}
0.037688 (0.053199) with: {'batch_size': 60, 'epochs': 10}
0.037688 (0.053199) with: {'batch_size': 60, 'epochs': 50}
0.037688 (0.053199) with: {'batch_size': 60, 'epochs': 100}
0.037688 (0.053199) with: {'batch_size': 80, 'epochs': 10}
0.037688 (0.053199) with: {'batch_size': 80, 'epochs': 50}
0.037688 (0.053199) with: {'batch_size': 80, 'epochs': 100}
0.030151 (0.042559) with: {'batch_size': 100, 'epochs':

In [20]:
from keras.optimizers import SGD 

In [21]:
def create_model(learn_rate=0.01, momentum=0):
    model = Sequential()
    model.add(Dense(25, input_dim=8, activation='relu')) # Hidden 1
    model.add(Dense(10, activation='relu')) # Hidden 2
    model.add(Dense(1)) # Output
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model 
                  

In [22]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3) 

In [ ]:
grid_result = grid.fit(x, y) 